### Importamos las librerias necesarias para realizar nuestro ETL:

In [1]:
from pyspark.sql.functions import col, substring, length, countDistinct
from pyspark.sql import SparkSession

### Creacion de tabla `processed_files_reviews_yelp_silver` y `processed_file_business_yelp_silver`para mantener el registro de los archivos ya procesados.

In [3]:
spark.sql("CREATE TABLE IF NOT EXISTS processed_files_reviews_yelp_silver (file_name STRING) USING DELTA")
spark.sql("CREATE TABLE IF NOT EXISTS processed_files_business_yelp_silver (file_name STRING) USING DELTA")

### Aca podes revisar las tablas que mantienen el registro de los archivos ya procesados.

In [2]:
spark.sql("SELECT * FROM processed_files_reviews_yelp_silver").show()

In [3]:
spark.sql("SELECT * FROM processed_files_business_yelp_silver").show()

### Obtenemos la lista de archivos contenidos en el storage. En este caso los contenidos en la carpeta silver que corresponden a los archivos luego del proceso de ETL.

In [4]:
# Crear una instancia de SparkSession
spark = SparkSession.builder.getOrCreate()

# Especificar el nombre del contenedor y directorio
container_name = "datumtech"
directory_path_reviews = "/silver/Yelpsilver/reviews-silver/"

# Obtener la lista de carpetas
adls_files_reviews = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration()) \
    .listStatus(spark._jvm.org.apache.hadoop.fs.Path(f"abfss://{container_name}.blob.core.windows.net/{directory_path_reviews}"))

In [5]:
#Directorio de business
directory_path_business = "/silver/Yelpsilver/business-silver/"

# Obtener la lista de carpetas
adls_files_business = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration()) \
    .listStatus(spark._jvm.org.apache.hadoop.fs.Path(f"abfss://{container_name}.blob.core.windows.net/{directory_path_business}"))

### Creamos una variable `new_files_reviews` y otra `new_files_business` que contiene los archivos que no estan procesados en ADLS.

In [6]:
processed_files_reviews = spark.sql("SELECT file_name FROM processed_files_reviews_yelp_silver").toPandas()["file_name"].tolist()

new_files_reviews = [file.getPath().getName() for file in adls_files_reviews if file.getPath().getName() not in processed_files_reviews]

In [7]:
processed_files_business = spark.sql("SELECT file_name FROM processed_files_business_yelp_silver").toPandas()["file_name"].tolist()

new_files_business = [file.getPath().getName() for file in adls_files_business if file.getPath().getName() not in processed_files_business]

### Podemos ver que archivos no estan procesados aun.

In [8]:
for file in new_files_reviews:
    print(file)

In [9]:
for file in new_files_business:
    print(file)

### Cargamos el archivo business correspondiente de la tabla silver para poder procesar las tablas reviews-silver.

In [4]:
path_business_silver = f"abfss://datumtech@datumlake.dfs.core.windows.net/silver/Yelpsilver/business-silver/{new_files_business[0]}"
df_business = spark.read.format("parquet").load(path_business_silver)

In [5]:
df_business.show(5)

In [6]:
df_business.dtypes

###

### Cambiamos el tipo de dato de la columna 'num_of_reviews' para evitar problemas al momento de la ingesta de los datos en la base de datos.

In [7]:
df_business = df_business.withColumn("review_count", col("review_count").cast("int"))

In [14]:
file_business_gold = new_files_business[0].rstrip("silver")

In [15]:
path_business_gold = f"abfss://datumtech@datumlake.dfs.core.windows.net/gold/Yelpgold/business-gold/{file_business_gold}gold"
df_business.write.format("parquet").save(path_business_gold)

In [16]:
df_business_gold = spark.read.format("parquet").load(path_business_gold)

In [17]:
df_business_gold.show(5)

In [18]:
unique_count = df_business_gold.select(countDistinct("business_id")).first()[0]
print("Número de valores únicos en la columna 'business_id':", unique_count)

### Creamos la funcion `gold` que realiza todo el proceso de agregacion de valor a la informacion y creacion de tablas secundarias para diferentes analisis y devuelve los dataframes en formato parquet a la tabla gold corrrespondiente a los datos ya procesados y enriquecidos.

In [8]:

def gold(file_reviews):
    
    # Definimos las rutas:
    file_reviews_gold = file_reviews.rstrip("silver")
    path_reviews_silver = f"abfss://datumtech@datumlake.dfs.core.windows.net/silver/Yelpsilver/reviews-silver/{file_reviews}"
    path_reviews_gold = f"abfss://datumtech@datumlake.dfs.core.windows.net/gold/Yelpgold/reviews-gold/{file_reviews_gold}gold"

    # Cargamos los archivo desde la tabla silver en el storage.
    df_reviews = spark.read.format("parquet").load(path_reviews_silver)
    
    # La columna "business_id" en df_business ya contiene valores únicos, filtraremos df_reviews basándonos en la columna "business_id":
    df_joined = df_reviews.join(df_business, on="business_id", how="inner")
    df_reviews = df_joined.select('review_id', 'user_id', 'business_id', 'stars', 'text', 'date')

    # Cambiamos el nombre de la columna stars a rating
    df_reviews = df_reviews.withColumnRenamed("stars", "rating")

    # Cambiamos el tipo de dato de la columna 'rating' para evitar problemas al momento de la ingesta de los datos en la base de datos.
    df_reviews = df_reviews.withColumn("rating", col("rating").cast("int"))

    # Truncamos la columna text a 7500 para evitar problema a la hora de cargar los datos en la base de datos.
    df_reviews = df_reviews.withColumn("text", substring(df_reviews["text"], 1, 7000))

    # Guardamos los DataFrames df_reviews y df_business en la tabla gold correspondiente a los datos procesados y enriquecidos.   
    return df_reviews.write.format("parquet").save(path_reviews_gold)
    

### Aplicamos la funcion `gold` a todos los archivos sin procesar:

In [ ]:
for file in new_files_reviews:
    gold(file)

#### Para comprobar que todo se haya ejecutado de manera correcta, podemos traer cualquier archivo de la tabla gold y hacer algunas verificaciones.

In [9]:
path_reviews_gold = f"abfss://datumtech@datumlake.dfs.core.windows.net/gold/Yelpgold/reviews-gold/review-gold"
df_reviews_gold = spark.read.format("parquet").load(path_reviews_gold)

In [10]:
df_reviews_gold.show(5)

In [24]:
df_reviews_gold.dtypes

In [30]:
df_reviews_gold.count()

### Agregamos a la tabla `processed_files_silver` y `processed_files_business_silver` los archivos ya procesados.

In [25]:
new_files_df = spark.createDataFrame([(file,) for file in new_files_reviews], ["file_name"])
new_files_df.write.format("delta").mode("append").saveAsTable("processed_files_reviews_yelp_silver")

In [26]:
new_files_df = spark.createDataFrame([(file,) for file in new_files_business], ["file_name"])
new_files_df.write.format("delta").mode("append").saveAsTable("processed_files_business_yelp_silver")

### Podemos verificar que, efectivamente esten registrados los archivos ya procesados.

In [28]:
spark.sql("SELECT * FROM processed_files_reviews_yelp_silver ").show()

In [29]:
spark.sql("SELECT * FROM processed_files_business_yelp_silver ").show()